In [8]:
from pathlib import Path

import mmtbx.f_model
import mmtbx.model
import cctbx.crystal
import cctbx.xray
import iotbx
from iotbx import pdb

import sys
sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
import miller_ops
sys.path.append(str(Path(Path.home(), "Documents/xray/data/cifs/scripts")))
import generate_fmodel

In [9]:
pdb_file = Path("~/Documents/xray/dev/26_phenix_refine/data/test.pdb")
n_state = 2

In [10]:
# crystal_symmetry = cctbx.xray.crystal.symmetry(
#     unit_cell=(114.968, 54.622, 45.194, 90.000, 101.675, 90.000),
#     space_group_symbol="C 1 2 1"
# )

pdb_inp = pdb.input(file_name=str(pdb_file))
h = pdb_inp.construct_hierarchy()
crystal_symmetry = pdb_inp.crystal_symmetry()

xray_structure = h.extract_xray_structure(crystal_symmetry=crystal_symmetry)

# xray_structure = iotbx.pdb.input(str(pdb_file)).xray_structure_simple(
#     crystal_symmetry=crystal_symmetry
# )
# xray_structure = read_non_altloc_structure(pdb_file)

n_scatt = xray_structure.scatterers().size()
n_scatt_per_state = int(n_scatt/n_state)

for i in range(n_state):
    for j in range(n_scatt_per_state):
        xray_structure.scatterers()[i*n_scatt_per_state+j].occupancy = 1/n_state

# f_model = xray_structure.structure_factors(
#     d_min=1.5
# ).f_calc().amplitudes()
# print(f_model.d_min())

# flags = f_model.generate_r_free_flags_basic()
# status = generate_fmodel.get_status_array(flags)


In [11]:
f_obs_file = Path(Path.home(), "Documents/xray/data/cifs/7mhf/7mhf.cif")
f_obs = miller_ops.get_miller_array(
    f_obs_file=f_obs_file,
    label="_refln.F_meas_au"
)
f_obs = miller_ops.clean_miller_array(f_obs)

# Set flags from file.
status_array = miller_ops.get_miller_array(
    f_obs_file=f_obs_file,
    label="_refln.status"
)
flags = status_array.customized_copy(data=status_array.data()=="f")
f_obs, flags = f_obs.common_sets(other=flags)

In [12]:
xray_structure.scatterers().flags_set_grads(
    state=False
)
xray_structure.scatterers().flags_set_grad_site(
    iselection=xray_structure.all_selection().iselection()
)
xray_structure.scatterers().flags_set_grad_occupancy(
    iselection=xray_structure.all_selection().iselection()
)

f_model_manager = mmtbx.f_model.manager(
    xray_structure=xray_structure,
    f_obs=f_obs,
    r_free_flags=flags,
    target_name="ml"
)

fmodels = mmtbx.fmodels(fmodel_xray=f_model_manager)
fmodels.update_xray_structure(
    xray_structure=xray_structure,
    update_f_calc=True
)

f_model_manager.update_all_scales(
    apply_scale_k1_to_f_obs=True,
    remove_outliers=False
)

work = f_model_manager.r_work()
free = f_model_manager.r_free()

print(f_model_manager.f_obs().size())
print(f_model_manager.f_model().size())


37901
37901


In [13]:
work, free

(0.20023079193021787, 0.22940496171191166)

In [21]:
fmodels_target_and_gradients = fmodels.target_and_gradients(compute_gradients=True)
score = fmodels_target_and_gradients.target()

print(score)

5.543418606252292


In [15]:
generate_fmodel.write_cif(f_model, status, Path(Path.home(), "Documents/xray/tmp/test.cif"))

In [42]:
# print(type(xray_structure))
# print(xray_structure.show_summary())
# print(xray_structure.show_scatterers())